# High-level LSTM Keras (CNTK) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from common.params_lstm import *
from common.utils import *

Using CNTK backend


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print(K.backend.backend())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.0.8
Numpy:  1.13.1
CNTK:  2.1
cntk


In [3]:
def create_symbol():
    model = Sequential()
    model.add(Embedding(MAXFEATURES, EMBEDSIZE, input_length=MAXLEN))
    model.add(GRU(NUMHIDDEN))
    model.add(Dense(2, activation='softmax'))
    return model

In [4]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.Adam(LR, BETA_1, BETA_2, EPS),
        metrics = ['accuracy'])
    return m

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 25000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000, 2) (25000, 2)
int32 int32 int32 int32
CPU times: user 5.9 s, sys: 260 ms, total: 6.16 s
Wall time: 7.27 s


In [6]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 240 ms, sys: 608 ms, total: 848 ms
Wall time: 613 ms


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 8 ms, sys: 40 ms, total: 48 ms
Wall time: 10.1 ms


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 150)          3750000   
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               75300     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 3,825,502
Trainable params: 3,825,502
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/3
25000/25000 [==============================] - 74s - loss: 0.4745 - acc: 0.7541    
Epoch 2/3
25000/25000 [==============================] - 74s - loss: 0.2188 - acc: 0.9179    - ETA: 5s -
Epoch 3/3
25000/25000 [==============================] - 74s - loss: 0.1189 - acc: 0.9606    
CPU times: user 3min 18s, sys: 26.3 s, total: 3min 44s
Wall time: 3min 43s


In [10]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 30.7 s, sys: 4.42 s, total: 35.1 s
Wall time: 35.2 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.85932
